<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

In [2]:
pip install tabulate


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np 
from tabulate import tabulate
import sqlite3
import os


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
df = pd.read_csv('/Users/avinashmacbookair/Documents/TREND Health Partners/Dataset/Healthcare_Dataset.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/avinashmacbookair/Documents/TREND Health Partners/Dataset/Healthcare_Dataset.csv'

In [ ]:
# Convert to datetime
df['treatment_start_date'] = pd.to_datetime(df['treatment_start_date'])
df['treatment_completion_date'] = pd.to_datetime(df['treatment_completion_date'])

# Extract date and time parts
df['treatment_start_date_only'] = df['treatment_start_date'].dt.date
df['treatment_start_time_only'] = df['treatment_start_date'].dt.time
df['treatment_end_date_only'] = df['treatment_completion_date'].dt.date
df['treatment_end_time_only'] = df['treatment_completion_date'].dt.time

# Drop original datetime columns
df.drop(columns=['treatment_start_date', 'treatment_completion_date'], inplace=True)

# Preview the updated DataFrame
df.head()


In [ ]:
df.head()

In [ ]:
# Convert to datetime once and store in a temp variable
outcome_datetime = pd.to_datetime(df['treatment_outcome_date'])

# Split into new date and time columns
df['treatment_outcome_date'] = outcome_datetime.dt.date
df['treatment_outcome_time'] = outcome_datetime.dt.time


In [ ]:
df.head()

In [ ]:
df.rename(columns={'treatment_start_date_only': 'treatment_start_date', 'treatment_start_time_only': 'treatment_start_time','treatment_end_date_only':'treatment_end_date','treatment_end_time_only':'treatment_end_time'}, inplace=True)
df.head()

In [ ]:
# Convert to datetime to ensure correct operations
df['treatment_outcome_date'] = pd.to_datetime(df['treatment_outcome_date'])
df['treatment_end_date'] = pd.to_datetime(df['treatment_end_date'])

# Add Outcome_Day column
df['Outcome_Day'] = df['treatment_outcome_date'].dt.day_name()

# Add Outcome_Weekend_Flag column (1 if Sat/Sun, else 0)
df['Outcome_Weekend_Flag'] = df['Outcome_Day'].isin(['Saturday', 'Sunday']).astype(int)

# Add Report_Duration column (difference in days)
df['Report_Duration'] = (df['treatment_outcome_date'] - df['treatment_end_date']).dt.days
df.head()

In [ ]:
df['Outcome_Quarter'] = pd.to_datetime(df['treatment_outcome_date']).dt.quarter
df.head()

In [ ]:
# Split provider_name into first and last name
df[['provider_first_name', 'provider_last_name']] = df['provider_name'].str.split(' ', n=1, expand=True)

# Split patient_name into first and last name
df[['patient_first_name', 'patient_last_name']] = df['patient_name'].str.split(' ', n=1, expand=True)


In [4]:
try:
    # Create a new SQLite database (or connect if it already exists)
    conn = sqlite3.connect('healthcare_data.db')
    cursor = conn.cursor()

    # You can add any table creation or setup code here if needed
    print("Database connection established successfully.")

    # Commit any changes (optional here if nothing to commit)
    conn.commit()

except sqlite3.Error as e:
    print("An error occurred while connecting to the database:", e)

finally:
    if conn:
        conn.close()
        print("Database connection closed.")



Database connection established successfully.
Database connection closed.


In [ ]:
try:
    # Reconnect to the database
    conn = sqlite3.connect('healthcare_data.db')
    cursor = conn.cursor()
    print("Connected to the database.")

    # 1. TREATMENT table (Fact Table)
    cursor.execute('DROP TABLE IF EXISTS TREATMENT;')

    cursor.execute('''
    CREATE TABLE TREATMENT (
        Treatment_ID INTEGER PRIMARY KEY,
        Start_Date TEXT,
        Completion_Date TEXT,
        Outcome_Date TEXT,
        Outcome_Quarter INTEGER,
        Treatment_Duration INTEGER,
        Cost REAL,
        Effectiveness_Score INTEGER,
        Type TEXT,
        Patient_ID INTEGER,
        Provider_ID INTEGER,
        Location_ID INTEGER,
        Disease_ID INTEGER,
        Outcome_Day TEXT,
        Outcome_Weekend_Flag INTEGER,
        Report_Duration INTEGER,
        FOREIGN KEY (Patient_ID) REFERENCES PATIENT(Patient_ID),
        FOREIGN KEY (Provider_ID) REFERENCES PROVIDER(Provider_ID),
        FOREIGN KEY (Location_ID) REFERENCES LOCATION(Location_ID),
        FOREIGN KEY (Disease_ID) REFERENCES DISEASE(Disease_ID)
    );
    ''')

    # 2. PATIENT table (Dimension Table)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS PATIENT (
        Patient_ID INTEGER PRIMARY KEY,
        First_Name TEXT,
        Last_Name TEXT,
        Name TEXT,
        Gender TEXT,
        Age INTEGER
    );
    ''')

    # 3. PROVIDER table (Dimension Table)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS PROVIDER (
        Provider_ID INTEGER PRIMARY KEY,
        First_Name TEXT,
        Last_Name TEXT,
        Speciality_Id INTEGER,
        Speciality_Name TEXT,
        Affiliated_Hospital TEXT
    );
    ''')

    # 4. DISEASE table (Dimension Table)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS DISEASE (
        Disease_ID INTEGER PRIMARY KEY,
        Speciality_Id INTEGER,
        Name TEXT,
        Type TEXT,
        Severity TEXT,
        Transmission_mode TEXT,
        Mortality_Rate REAL
    );
    ''')

    # 5. LOCATION table (Dimension Table)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS LOCATION (
        Location_ID INTEGER PRIMARY KEY,
        Country TEXT,
        State TEXT,
        City TEXT
    );
    ''')

    # 6. EFFECTIVENESS (Dimension Table)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS EFFECTIVENESS (
    Effectiveness_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Outcome_Status TEXT UNIQUE,
    Effectiveness_Score INTEGER
    );

    ''')

    # Commit the changes
    conn.commit()
    print("Tables created successfully.")

except sqlite3.Error as e:
    print("An error occurred:", e)

finally:
    if conn:
        conn.close()
        print("Database connection closed.")


In [ ]:

# Connect to your database (change path as needed)
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

# Step 2: Populate DIM_EFFECTIVENESS table
cursor.execute('''
CREATE TABLE IF NOT EXISTS EFFECTIVENESS (
    Effectiveness_ID INTEGER PRIMARY KEY,
    Outcome_Status TEXT UNIQUE,
    Effectiveness_Score INTEGER
    );
''')


effectiveness_mapping = {
    'deceased': 0,
    'worsened': 1,
    'unsuccessful': 2,
    'partially successful': 3,
    'stable': 4,
    'successful': 5
}

for idx, (status, score) in enumerate(effectiveness_mapping.items(), start=1):
    cursor.execute('''
        INSERT OR IGNORE INTO EFFECTIVENESS (Effectiveness_ID, Outcome_Status, Effectiveness_Score)
        VALUES (?, ?, ?)
    ''', (idx, status, score))


# Save and close
conn.commit()
conn.close()

print("Lookup table populated and TREATMENT table updated.")


In [ ]:
# Connect to database
conn = sqlite3.connect('healthcare_data.db')
cursor = conn.cursor()

# Track unique location combinations to avoid duplicates
unique_locations = set()

for _, row in df.iterrows():
    # Insert into PATIENT using unique Patient_ID
    cursor.execute('''
    INSERT OR IGNORE INTO PATIENT (Patient_ID, First_Name, Last_Name, Gender, Age)
    VALUES (?, ?, ?, ?, ?)
    ''', (row['patient_id'], row['patient_first_name'], row['patient_last_name'], row['gender'], row['age']))

    # Insert into PROVIDER using unique Provider_ID
    cursor.execute('''
    INSERT OR IGNORE INTO PROVIDER (Provider_ID, First_Name, Last_Name, Speciality_Id, Speciality_Name, Affiliated_Hospital)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (row['provider_id'], row['provider_first_name'], row['provider_last_name'], row['speciality_id_x'], row['speciality_name'], row['affiliated_hospital']))

    # Insert into DISEASE using unique Disease_ID
    cursor.execute('''
    INSERT OR IGNORE INTO DISEASE (Disease_ID, Speciality_Id, Name, Type, Severity, Transmission_Mode, Mortality_Rate)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (row['disease_id'], row['speciality_id_x'], row['disease_name'], row['disease_type'], row['severity'], row['transmission_mode'], row['mortality_rate']))

    # Handle LOCATION uniqueness by (Country, State, City) combination
    location_key = (row['country'], row['state'], row['city'])
    if location_key not in unique_locations:
        cursor.execute('''
        INSERT INTO LOCATION (Location_ID, Country, State, City)
        VALUES (?, ?, ?, ?)
        ''', (row['location_id'], row['country'], row['state'], row['city']))
        unique_locations.add(location_key)

    effectiveness_score = effectiveness_mapping.get(row['treatment_outcome_status'].lower(), None)

    # Insert into TREATMENT (fact table, always insert)
    cursor.execute('''
    INSERT INTO TREATMENT (
        Treatment_ID, Start_Date, Completion_Date, Outcome_Date, Outcome_Quarter, Treatment_Duration, Cost, 
        Effectiveness_Score, Type, Patient_ID, Provider_ID, Location_ID, Disease_ID,
        Outcome_Day, Outcome_Weekend_Flag, Report_Duration
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        row['treatment_id'],
        row['treatment_start_date'].strftime('%Y-%m-%d') if pd.notnull(row['treatment_start_date']) else None,
        row['treatment_end_date'].strftime('%Y-%m-%d') if pd.notnull(row['treatment_end_date']) else None,
        row['treatment_outcome_date'].strftime('%Y-%m-%d') if pd.notnull(row['treatment_outcome_date']) else None,
        row['Outcome_Quarter'],
        row['treatment_duration'],
        row['treatment_cost'],
        #row['treatment_outcome_status'],
        effectiveness_score,
        row['treatment_type'],
        row['patient_id'],
        row['provider_id'],
        row['location_id'],
        row['disease_id'],
        row['Outcome_Day'],
        row['Outcome_Weekend_Flag'],
        row['Report_Duration']
    ))

# Commit changes and close the connection
conn.commit()
conn.close()


In [ ]:
# Connect to the database
conn = sqlite3.connect('healthcare_data.db')

# Run your query and load into DataFrame
query = 'SELECT * FROM TREATMENT LIMIT 5;'
df_result = pd.read_sql_query(query, conn)

# Print in nice table format
print(tabulate(df_result, headers='keys', tablefmt='psql'))

# Close the connection
conn.close()

In [5]:
# Specify the database file path
db_file = 'healthcare_data.db'

# Check if the database file exists
if os.path.exists(db_file):
    os.remove(db_file)
    print(f"Database {db_file} deleted successfully.")
else:
    print(f"Database {db_file} does not exist.")


Database healthcare_data.db deleted successfully.
